### data ###

In [ ]:
import zipfile
import os
import cv2

def unzip_file(zip_path, extract_to='.'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Unzip manually downloaded files
unzip_file('WIDER_train.zip', 'wider_face/')
unzip_file('WIDER_val.zip', 'wider_face/')


In [ ]:
from pathlib import Path

for split in ['train', 'val']:
    Path(f'datasets/face/images/{split}').mkdir(parents=True, exist_ok=True)
    Path(f'datasets/face/labels/{split}').mkdir(parents=True, exist_ok=True)


### **Dataset Preparation**

In [ ]:
def process_wider_annotations(image_dir, annotation_file, output_label_dir, output_img_dir):

    os.makedirs(output_label_dir, exist_ok=True)
    os.makedirs(output_img_dir, exist_ok=True)

    with open(annotation_file, 'r') as f:
        lines = f.readlines()

    idx = 0
    while idx < len(lines):
        filename = lines[idx].strip()
        idx += 1

        if idx >= len(lines):
            break  

        try:
            num_faces = int(lines[idx].strip())
        except ValueError:
            print(f"⚠️ Skipped invalid line: {lines[idx].strip()}")
            break

        idx += 1

        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(output_label_dir, os.path.splitext(filename)[0] + '.txt')
        label_dir = os.path.dirname(label_path)
        os.makedirs(label_dir, exist_ok=True)

        output_img_path = os.path.join(output_img_dir, filename)
        output_img_dirname = os.path.dirname(output_img_path)
        os.makedirs(output_img_dirname, exist_ok=True)
        if os.path.exists(image_path):
            cv2.imwrite(output_img_path, cv2.imread(image_path))
        else:
            print(f"🚫 Image not found: {image_path}")
            continue

        with open(label_path, 'w') as label_file:
            for _ in range(num_faces):
                if idx >= len(lines):
                    break  
                box = list(map(int, lines[idx].strip().split()))
                x, y, w, h = box[:4]
                cx = x + w / 2
                cy = y + h / 2
                label_file.write(f"0 {cx} {cy} {w} {h}\n")  
                idx += 1


# Run conversion
process_wider_annotations(
    image_dir='wider_face/WIDER_train/images',
    annotation_file='wider_face/wider_face_split/wider_face_train_bbx_gt.txt',
    output_label_dir='datasets/face/labels/train',
    output_img_dir='datasets/face/images/train'
)

process_wider_annotations(
    image_dir='wider_face/WIDER_val/images',
    annotation_file='wider_face/wider_face_split/wider_face_val_bbx_gt.txt',
    output_label_dir='datasets/face/labels/val',
    output_img_dir='datasets/face/images/val'
)


In [ ]:
yaml_content = """
path: datasets/face
train: images/train
val: images/val

nc: 1
names: ['face']
"""

with open("face.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')

model.model.fuse()

for param in list(model.model.parameters())[:-10]:
    param.requires_grad = False


# Start training
model.train(
    data='face.yaml',
    epochs=15,
    imgsz=640,
    batch=16,
    name='yolov8_face'
)


In [ ]:
import cv2
import random

# Load your fine-tuned model
model = YOLO("runs/detect/yolov8_face/weights/best.pt")

# Load the uploaded image
image_path = "face_img.jpg"
image = cv2.imread(image_path)

# Predict with low confidence threshold
results = model(image_path, conf=0.1)[0]

# Show number of detected faces
print(f"Detected {len(results.boxes)} faces")

# Draw a random color box around each detected face
for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    conf = float(box.conf[0])
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, f"{conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Save the output image
output_path = "predicted_faces.jpg"
cv2.imwrite(output_path, image)
print(f"✅ Saved output to: {output_path}")
